In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/predict-the-success-of-bank-telemarketing/sample_submission.csv
/kaggle/input/predict-the-success-of-bank-telemarketing/train.csv
/kaggle/input/predict-the-success-of-bank-telemarketing/test.csv


**Predict the Success of Bank Telemarketing**

Project Description:
This project aims to predict whether a client will subscribe to a bank's term deposit based on data from direct marketing campaigns.

**Dummy model submission**

In [2]:
# df_dummy = pd.read_csv("/kaggle/input/predict-the-success-of-bank-telemarketing/train.csv")
# x_dummy = df_dummy.drop('target',axis=1)
# y_dummy = df_dummy['target']
# from sklearn.dummy import DummyClassifier
# DummyModel = DummyClassifier().fit(x_dummy,y_dummy)
# x_dummy_test = pd.read_csv("/kaggle/input/predict-the-success-of-bank-telemarketing/test.csv")
# y_dummy_pred = DummyModel.predict(x_dummy_test)
# submission = pd.DataFrame({"id":range(0,x_dummy_test.shape[0]),'target':y_dummy_pred})
# submission.to_csv("submission.csv",index = False)

In [3]:
#IMPORTS
from sklearn.pipeline import Pipeline

from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler,LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV

from sklearn.linear_model import LogisticRegression, Perceptron, RidgeClassifier,SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier


from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import f1_score



In [4]:
train_dataset =  pd.read_csv("/kaggle/input/predict-the-success-of-bank-telemarketing/train.csv")
print("Missing values in the train dataset:")
missing_values = train_dataset.isnull().sum()
print(missing_values)

Missing values in the train dataset:
last contact date        0
age                      0
job                    229
marital                  0
education             1467
default                  0
balance                  0
housing                  0
loan                     0
contact              10336
duration                 0
campaign                 0
pdays                    0
previous                 0
poutcome             29451
target                   0
dtype: int64


In [5]:
test_dataset =  pd.read_csv("/kaggle/input/predict-the-success-of-bank-telemarketing/test.csv")

print("Missing values in the test dataset:")
missing_values = test_dataset.isnull().sum()
print(missing_values)

Missing values in the test dataset:
last contact date       0
age                     0
job                    59
marital                 0
education             390
default                 0
balance                 0
housing                 0
loan                    0
contact              2684
duration                0
campaign                0
pdays                   0
previous                0
poutcome             7508
dtype: int64


**EDA on Train Dataset**

Loading the train dataset

In [6]:
train_dataset =  pd.read_csv("/kaggle/input/predict-the-success-of-bank-telemarketing/train.csv")

train_dataset.head()

,last contact date,age,job,marital,education,default,balance,housing,loan,contact,duration,campaign,pdays,previous,poutcome,target
0,2009-04-17,26,blue-collar,married,secondary,no,647,yes,no,cellular,357,2,331,1,other,no
1,2009-10-11,52,technician,married,secondary,no,553,yes,no,telephone,160,1,-1,0,NaN,no
2,2010-11-20,44,blue-collar,married,secondary,no,1397,no,no,cellular,326,1,-1,0,NaN,no
3,2009-09-01,33,admin.,married,secondary,no,394,yes,no,telephone,104,3,-1,0,NaN,no
4,2008-01-29,31,entrepreneur,single,tertiary,no,137,no,no,cellular,445,2,-1,0,NaN,no


In [7]:
# Check the shape of the dataset
print(f"\nTraining dataset has {train_dataset.shape[0]} rows and {train_dataset.shape[1]} columns.")


Training dataset has 39211 rows and 16 columns.


In [8]:
train_dataset.describe()

,age,balance,duration,campaign,pdays,previous
count,39211.000000,39211.000000,39211.000000,39211.000000,39211.000000,39211.000000
mean,42.120247,5441.781719,439.062789,5.108770,72.256051,11.826171
std,12.709352,16365.292065,769.096291,9.890153,160.942593,44.140259
min,18.000000,-8019.000000,0.000000,1.000000,-1.000000,0.000000
25%,33.000000,98.000000,109.000000,1.000000,-1.000000,0.000000
50%,40.000000,549.000000,197.000000,2.000000,-1.000000,0.000000
75%,50.000000,2030.000000,380.000000,4.000000,-1.000000,0.000000
max,95.000000,102127.000000,4918.000000,63.000000,871.000000,275.000000


In [9]:
train_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39211 entries, 0 to 39210
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   last contact date  39211 non-null  object
 1   age                39211 non-null  int64 
 2   job                38982 non-null  object
 3   marital            39211 non-null  object
 4   education          37744 non-null  object
 5   default            39211 non-null  object
 6   balance            39211 non-null  int64 
 7   housing            39211 non-null  object
 8   loan               39211 non-null  object
 9   contact            28875 non-null  object
 10  duration           39211 non-null  int64 
 11  campaign           39211 non-null  int64 
 12  pdays              39211 non-null  int64 
 13  previous           39211 non-null  int64 
 14  poutcome           9760 non-null   object
 15  target             39211 non-null  object
dtypes: int64(6), object(10)
memory usage: 4.

In [10]:
# Check for missing values
print("Missing values in the training dataset:")
missing_values = train_dataset.isnull().sum()
print(missing_values)

Missing values in the training dataset:
last contact date        0
age                      0
job                    229
marital                  0
education             1467
default                  0
balance                  0
housing                  0
loan                     0
contact              10336
duration                 0
campaign                 0
pdays                    0
previous                 0
poutcome             29451
target                   0
dtype: int64


**There is no missing value in target collum so we can't drop any row with missing value**

**The missing values are present at columns ['job', 'education','contact','poutcome']**  of which **all are categorical value**

In [11]:
print(train_dataset.dtypes)

last contact date    object
age                   int64
job                  object
marital              object
education            object
default              object
balance               int64
housing              object
loan                 object
contact              object
duration              int64
campaign              int64
pdays                 int64
previous              int64
poutcome             object
target               object
dtype: object


In [12]:
#categorical columns and unique values
categorical_cols = train_dataset.select_dtypes(include=['object']).columns

print(f"Categorical columns in the dataset: {list(categorical_cols)}")


print("\nUnique values in each categorical column:")
for col in categorical_cols:
    print(f"\n{col}:")
    print(train_dataset[col].unique())
#     print(train_df[col].value_counts())


Categorical columns in the dataset: ['last contact date', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'poutcome', 'target']

Unique values in each categorical column:

last contact date:
['2009-04-17' '2009-10-11' '2010-11-20' ... '2009-07-05' '2008-02-25'
 '2010-04-12']

job:
['blue-collar' 'technician' 'admin.' 'entrepreneur' 'unemployed'
 'management' 'services' 'student' 'self-employed' 'retired' 'housemaid'
 nan]

marital:
['married' 'single' 'divorced']

education:
['secondary' 'tertiary' 'primary' nan]

default:
['no' 'yes']

housing:
['yes' 'no']

loan:
['no' 'yes']

contact:
['cellular' 'telephone' nan]

poutcome:
['other' nan 'failure' 'success']

target:
['no' 'yes']


all missing value shown as **nan**

In [13]:
#  numerical columns
numerical_cols = train_dataset.select_dtypes(include=['int64', 'float64']).columns

print(f"Numerical columns in the dataset: {list(numerical_cols)}")

# Summary statistics 
print("\nSummary statistics for numerical columns:")
print(train_dataset[numerical_cols].describe())




Numerical columns in the dataset: ['age', 'balance', 'duration', 'campaign', 'pdays', 'previous']

Summary statistics for numerical columns:
                age        balance      duration      campaign         pdays  \
count  39211.000000   39211.000000  39211.000000  39211.000000  39211.000000   
mean      42.120247    5441.781719    439.062789      5.108770     72.256051   
std       12.709352   16365.292065    769.096291      9.890153    160.942593   
min       18.000000   -8019.000000      0.000000      1.000000     -1.000000   
25%       33.000000      98.000000    109.000000      1.000000     -1.000000   
50%       40.000000     549.000000    197.000000      2.000000     -1.000000   
75%       50.000000    2030.000000    380.000000      4.000000     -1.000000   
max       95.000000  102127.000000   4918.000000     63.000000    871.000000   

           previous  
count  39211.000000  
mean      11.826171  
std       44.140259  
min        0.000000  
25%        0.000000  
50%   

In [14]:
print(train_dataset[numerical_cols].corr())

               age   balance  duration  campaign     pdays  previous
age       1.000000  0.289150  0.255573  0.269439  0.199507  0.277487
balance   0.289150  1.000000  0.669737  0.674339  0.562097  0.716816
duration  0.255573  0.669737  1.000000  0.630944  0.527020  0.683951
campaign  0.269439  0.674339  0.630944  1.000000  0.519469  0.689211
pdays     0.199507  0.562097  0.527020  0.519469  1.000000  0.586175
previous  0.277487  0.716816  0.683951  0.689211  0.586175  1.000000


* Account balance is strongly correlated with previous contacts and moderately correlated with both campaign and duration. This suggests that clients with higher account balances tend to have longer interactions and more frequent previous contacts.

* Duration and previous contacts have a strong positive correlation, indicating that clients with a history of frequent interactions tend to engage in longer conversations during the current campaign.

*  campaign is moderately correlated with both balance and duration, suggesting that clients with higher balances and longer interactions in the past are targeted more heavily in the current campaign.

* Age shows weak relationships with other features, suggesting that age is not a strong predictor for most of these interactions, though there may be a slight tendency for older clients to have higher balances and longer contact durations.

no column are neglegible

**Feature Engineering**

suspecting that 'last contact date' may play a role in prediction so i convert the object in datetime object also separating the dates into year, month, day column

In [15]:

train_dataset['last contact date'] = pd.to_datetime(train_dataset['last contact date'], errors='coerce')

train_dataset['year'] = train_dataset['last contact date'].dt.year
train_dataset['month'] = train_dataset['last contact date'].dt.month
train_dataset['day'] = train_dataset['last contact date'].dt.day




# # Check the updated DataFrame
print(train_dataset.head())


  last contact date  age           job  marital  education default  balance  \
0        2009-04-17   26   blue-collar  married  secondary      no      647   
1        2009-10-11   52    technician  married  secondary      no      553   
2        2010-11-20   44   blue-collar  married  secondary      no     1397   
3        2009-09-01   33        admin.  married  secondary      no      394   
4        2008-01-29   31  entrepreneur   single   tertiary      no      137   

  housing loan    contact  duration  campaign  pdays  previous poutcome  \
0     yes   no   cellular       357         2    331         1    other   
1     yes   no  telephone       160         1     -1         0      NaN   
2      no   no   cellular       326         1     -1         0      NaN   
3     yes   no  telephone       104         3     -1         0      NaN   
4      no   no   cellular       445         2     -1         0      NaN   

  target  year  month  day  
0     no  2009      4   17  
1     no  2009  

In [16]:
 # droping last contact date column
train_dataset = train_dataset.drop(columns=['last contact date']) 
print(train_dataset.head())

   age           job  marital  education default  balance housing loan  \
0   26   blue-collar  married  secondary      no      647     yes   no   
1   52    technician  married  secondary      no      553     yes   no   
2   44   blue-collar  married  secondary      no     1397      no   no   
3   33        admin.  married  secondary      no      394     yes   no   
4   31  entrepreneur   single   tertiary      no      137      no   no   

     contact  duration  campaign  pdays  previous poutcome target  year  \
0   cellular       357         2    331         1    other     no  2009   
1  telephone       160         1     -1         0      NaN     no  2009   
2   cellular       326         1     -1         0      NaN     no  2010   
3  telephone       104         3     -1         0      NaN     no  2009   
4   cellular       445         2     -1         0      NaN     no  2008   

   month  day  
0      4   17  
1     10   11  
2     11   20  
3      9    1  
4      1   29  


In [17]:
#droping target column
df_features = train_dataset.drop(columns = ['target'])
targetlabel= train_dataset['target']

In [18]:
print(df_features.columns)
print(targetlabel.head())

Index(['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'contact', 'duration', 'campaign', 'pdays', 'previous',
       'poutcome', 'year', 'month', 'day'],
      dtype='object')
0    no
1    no
2    no
3    no
4    no
Name: target, dtype: object


In [19]:
targetlabel.info()

<class 'pandas.core.series.Series'>
RangeIndex: 39211 entries, 0 to 39210
Series name: target
Non-Null Count  Dtype 
--------------  ----- 
39211 non-null  object
dtypes: object(1)
memory usage: 306.5+ KB


In [20]:
print(df_features.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39211 entries, 0 to 39210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        39211 non-null  int64 
 1   job        38982 non-null  object
 2   marital    39211 non-null  object
 3   education  37744 non-null  object
 4   default    39211 non-null  object
 5   balance    39211 non-null  int64 
 6   housing    39211 non-null  object
 7   loan       39211 non-null  object
 8   contact    28875 non-null  object
 9   duration   39211 non-null  int64 
 10  campaign   39211 non-null  int64 
 11  pdays      39211 non-null  int64 
 12  previous   39211 non-null  int64 
 13  poutcome   9760 non-null   object
 14  year       39211 non-null  int32 
 15  month      39211 non-null  int32 
 16  day        39211 non-null  int32 
dtypes: int32(3), int64(6), object(8)
memory usage: 4.6+ MB
None


**Data preprocessing**

The missing values are present at columns ['job', 'education','contact','poutcome'] of which all are categorical value.

missing value is 'nan'


* The imputation process replaces missing values in job, education, contact, and poutcome with their respective most frequent values.
* The use of ColumnTransformer ensures that only these specific columns are imputed, and all other columns are left unchanged.
* The resulting DataFrame (df_imputed) has the same structure as the original, with missing values in the specified columns replaced.

In [21]:
categorical_columns_with_missing = ['job', 'education', 'contact', 'poutcome']


ct1 = ColumnTransformer([
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='most_frequent'), categorical_columns_with_missing)
],
    remainder='passthrough',  #column is not listed in  will be left unchanged.
    verbose_feature_names_out=False #keeps the original column names.
).set_output(transform='pandas')  #output remains a Pandas DataFrame (instead of a NumPy array).


df_imputed = ct1.fit_transform(df_features)


print(df_imputed.shape)

(39211, 17)


In [22]:
#categorical columns
categorical_cols = df_imputed.select_dtypes(include=['object']).columns

print(f"Categorical columns in the dataset: {list(categorical_cols)}")


print("\nUnique values in each categorical column:")
for col in categorical_cols:
    print(f"\n{col}:")
    print(df_imputed[col].unique())

Categorical columns in the dataset: ['job', 'education', 'contact', 'poutcome', 'marital', 'default', 'housing', 'loan']

Unique values in each categorical column:

job:
['blue-collar' 'technician' 'admin.' 'entrepreneur' 'unemployed'
 'management' 'services' 'student' 'self-employed' 'retired' 'housemaid']

education:
['secondary' 'tertiary' 'primary']

contact:
['cellular' 'telephone']

poutcome:
['other' 'failure' 'success']

marital:
['married' 'single' 'divorced']

default:
['no' 'yes']

housing:
['yes' 'no']

loan:
['no' 'yes']


In [23]:
df_imputed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39211 entries, 0 to 39210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   job        39211 non-null  object
 1   education  39211 non-null  object
 2   contact    39211 non-null  object
 3   poutcome   39211 non-null  object
 4   age        39211 non-null  int64 
 5   marital    39211 non-null  object
 6   default    39211 non-null  object
 7   balance    39211 non-null  int64 
 8   housing    39211 non-null  object
 9   loan       39211 non-null  object
 10  duration   39211 non-null  int64 
 11  campaign   39211 non-null  int64 
 12  pdays      39211 non-null  int64 
 13  previous   39211 non-null  int64 
 14  year       39211 non-null  int32 
 15  month      39211 non-null  int32 
 16  day        39211 non-null  int32 
dtypes: int32(3), int64(6), object(8)
memory usage: 4.6+ MB


Now no missing value at any columns

In [24]:
from scipy.stats import chi2_contingency
categorical_cols = df_imputed.select_dtypes(include=['object']).columns.tolist()

for col1 in categorical_cols:
    for col2 in categorical_cols:
        if col1 != col2:  
            contingency_table = pd.crosstab(df_imputed[col1], df_imputed[col2])
            
            # Perform Chi-squared test
            chi2, p, dof, expected = chi2_contingency(contingency_table)
          
            print(f"Chi-squared Test between {col1} and {col2}:")
            print(f"Chi-squared statistic: {chi2:.4f}")
            print(f"P-value: {p:.4f}")
            
            if p < 0.05:
                print(f"There's a significant association between {col1} and {col2}\n")
            else:
                print(f"There's no significant association between {col1} and {col2}\n")

Chi-squared Test between job and education:
Chi-squared statistic: 20620.1411
P-value: 0.0000
There's a significant association between job and education

Chi-squared Test between job and contact:
Chi-squared statistic: 2649.2641
P-value: 0.0000
There's a significant association between job and contact

Chi-squared Test between job and poutcome:
Chi-squared statistic: 3590.8803
P-value: 0.0000
There's a significant association between job and poutcome

Chi-squared Test between job and marital:
Chi-squared statistic: 2854.0054
P-value: 0.0000
There's a significant association between job and marital

Chi-squared Test between job and default:
Chi-squared statistic: 4347.6449
P-value: 0.0000
There's a significant association between job and default

Chi-squared Test between job and housing:
Chi-squared statistic: 2452.9233
P-value: 0.0000
There's a significant association between job and housing

Chi-squared Test between job and loan:
Chi-squared statistic: 661.6207
P-value: 0.0000
There'

All the pairs of variables  tested  show significant associations, as indicated by the p-values of 0.0000 or values extremely close to zero. This suggests that there is a statistically significant relationship between each pair of variables.

we can't drop any column

In [25]:

categorical_cols = df_imputed.select_dtypes(include=['object']).columns.tolist()
numerical_cols = df_imputed.select_dtypes(include=['int64', 'float64','int32']).columns.tolist()


print("Categorical Columns:", categorical_cols)
print("Numerical Columns:", numerical_cols)


Categorical Columns: ['job', 'education', 'contact', 'poutcome', 'marital', 'default', 'housing', 'loan']
Numerical Columns: ['age', 'balance', 'duration', 'campaign', 'pdays', 'previous', 'year', 'month', 'day']


In [26]:
categorical_cols = df_features.select_dtypes(include=['object']).columns.tolist()


numerical_cols = df_features.select_dtypes(include=['int64', 'float64', 'int32']).columns.tolist()


full_pipeline = Pipeline([
    ('encoding_and_scaling', ColumnTransformer([
        ('ohe', OneHotEncoder(sparse_output=False), categorical_cols),  # One-hot encode categorical columns
        ('stdscl', StandardScaler(), numerical_cols)  # Standardize numerical columns
    ], remainder='passthrough', verbose_feature_names_out=False).set_output(transform='pandas'))
])


df_pre_train = full_pipeline.fit_transform(df_imputed)


df_pre_train.shape

(39211, 37)

In [27]:
print(df_pre_train.columns)


Index(['job_admin.', 'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'marital_divorced',
       'marital_married', 'marital_single', 'education_primary',
       'education_secondary', 'education_tertiary', 'default_no',
       'default_yes', 'housing_no', 'housing_yes', 'loan_no', 'loan_yes',
       'contact_cellular', 'contact_telephone', 'poutcome_failure',
       'poutcome_other', 'poutcome_success', 'age', 'balance', 'duration',
       'campaign', 'pdays', 'previous', 'year', 'month', 'day'],
      dtype='object')


In [28]:
df_pre_train.shape

(39211, 37)

In [29]:
#encoding targetlabel
# print(type(targetlabel))

label_encoder = LabelEncoder()

target_encoded = label_encoder.fit_transform(targetlabel)
target_encoded_train = pd.Series(target_encoded, name="target_encoded")
# print(type(target_encoded_train))

target_encoded_train.shape

(39211,)

**Preprocessing test dataset**

In [30]:
test_dataset =  pd.read_csv("/kaggle/input/predict-the-success-of-bank-telemarketing/test.csv")

print("Missing values in the test dataset:")
missing_values = test_dataset.isnull().sum()
print(missing_values)

Missing values in the test dataset:
last contact date       0
age                     0
job                    59
marital                 0
education             390
default                 0
balance                 0
housing                 0
loan                    0
contact              2684
duration                0
campaign                0
pdays                   0
previous                0
poutcome             7508
dtype: int64


In [31]:
test_dataset.columns

Index(['last contact date', 'age', 'job', 'marital', 'education', 'default',
       'balance', 'housing', 'loan', 'contact', 'duration', 'campaign',
       'pdays', 'previous', 'poutcome'],
      dtype='object')

In [32]:
test_dataset['last contact date'] = pd.to_datetime(test_dataset['last contact date'], errors='coerce')

test_dataset['year'] = test_dataset['last contact date'].dt.year
test_dataset['month'] = test_dataset['last contact date'].dt.month
test_dataset['day'] = test_dataset['last contact date'].dt.day




# # Check the updated DataFrame
print(test_dataset.head())


  last contact date  age          job  marital  education default  balance  \
0        2009-11-21   36   management   single   tertiary      no        7   
1        2010-02-04   30   unemployed  married   tertiary      no     1067   
2        2010-07-28   32  blue-collar   single  secondary      no       82   
3        2010-06-09   38       admin.  married    primary      no     1487   
4        2008-03-02   59   management  married   tertiary      no      315   

  housing loan   contact  duration  campaign  pdays  previous poutcome  year  \
0      no   no       NaN        20         1     -1         0      NaN  2009   
1      no   no  cellular        78         2     -1         0      NaN  2010   
2     yes   no  cellular        86         4     -1         0      NaN  2010   
3      no   no       NaN       332         2     -1         0      NaN  2010   
4      no   no  cellular       591         1    176         2  failure  2008   

   month  day  
0     11   21  
1      2    4  
2 

In [33]:
test_dataset = test_dataset.drop(columns=['last contact date'])
print(test_dataset.head())

   age          job  marital  education default  balance housing loan  \
0   36   management   single   tertiary      no        7      no   no   
1   30   unemployed  married   tertiary      no     1067      no   no   
2   32  blue-collar   single  secondary      no       82     yes   no   
3   38       admin.  married    primary      no     1487      no   no   
4   59   management  married   tertiary      no      315      no   no   

    contact  duration  campaign  pdays  previous poutcome  year  month  day  
0       NaN        20         1     -1         0      NaN  2009     11   21  
1  cellular        78         2     -1         0      NaN  2010      2    4  
2  cellular        86         4     -1         0      NaN  2010      7   28  
3       NaN       332         2     -1         0      NaN  2010      6    9  
4  cellular       591         1    176         2  failure  2008      3    2  


In [34]:
test_dataset.columns

Index(['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'contact', 'duration', 'campaign', 'pdays', 'previous',
       'poutcome', 'year', 'month', 'day'],
      dtype='object')

In [35]:
test_dataset.shape

(10000, 17)

In [36]:

test_imputed = ct1.fit_transform(test_dataset)


print(test_imputed.head())
test_imputed.shape

           job  education   contact poutcome  age  marital default  balance  \
0   management   tertiary  cellular  failure   36   single      no        7   
1   unemployed   tertiary  cellular  failure   30  married      no     1067   
2  blue-collar  secondary  cellular  failure   32   single      no       82   
3       admin.    primary  cellular  failure   38  married      no     1487   
4   management   tertiary  cellular  failure   59  married      no      315   

  housing loan  duration  campaign  pdays  previous  year  month  day  
0      no   no        20         1     -1         0  2009     11   21  
1      no   no        78         2     -1         0  2010      2    4  
2     yes   no        86         4     -1         0  2010      7   28  
3      no   no       332         2     -1         0  2010      6    9  
4      no   no       591         1    176         2  2008      3    2  


(10000, 17)

In [37]:
print("Missing values in the test dataset:")
missing_values = test_imputed.isnull().sum()
print(missing_values)

Missing values in the test dataset:
job          0
education    0
contact      0
poutcome     0
age          0
marital      0
default      0
balance      0
housing      0
loan         0
duration     0
campaign     0
pdays        0
previous     0
year         0
month        0
day          0
dtype: int64


In [38]:
#encoding and scaling
test_pre = full_pipeline.fit_transform(test_imputed)

test_pre.head()

,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,...,poutcome_success,age,balance,duration,campaign,pdays,previous,year,month,day
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-0.494976,-0.333785,-0.547978,-0.416995,-0.453373,-0.270508,-0.001592,1.814381,0.638344
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-0.969303,-0.267302,-0.471782,-0.316748,-0.453373,-0.270508,1.223307,-1.603104,-1.405395
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-0.811194,-0.329081,-0.461272,-0.116256,-0.453373,-0.270508,1.223307,0.295499,1.479883
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-0.336867,-0.240959,-0.138096,-0.316748,-0.453373,-0.270508,1.223307,-0.084222,-0.804295
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.323277,-0.314467,0.202159,-0.416995,0.650351,-0.225528,-1.226492,-1.223384,-1.645835


In [39]:
test_pre.shape

(10000, 37)

****Now all preprocessing is done 
the traindataset is 
                   feattures = df_pre_train,
                   target    = target_encoded_train
                   
             test dataset 
                  features  = test_pre
              
              
 **target label encoded with **
 
 label_encoder = LabelEncoder()
 target_encoded = label_encoder.fit_transform(targetlabel)

In [40]:
print(df_pre_train.shape)
print(target_encoded_train.shape)
print(test_pre.shape)

(39211, 37)
(39211,)
(10000, 37)


Spliting train dataset for training and testing

In [41]:
X_train, X_test, y_train, y_test = train_test_split(df_pre_train, target_encoded_train, test_size=0.2, random_state=42)

# Check the shape of the train and test sets
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((31368, 37), (7843, 37), (31368,), (7843,))

here i am using 80% of data for training

**Model building**

In [42]:
#LogisticRegression

model1 = LogisticRegression(max_iter=1000, random_state=42) #maximum number of iterations for the solver to converge to a solution
# results are reproducible.

model1.fit(X_train, y_train)

y1_pred = model1.predict(X_test)


accuracy = accuracy_score(y_test, y1_pred)
print(f'Accuracy: {accuracy:.4f}')
print('Confusion Matrix:')
print(confusion_matrix(y_test, y1_pred))
print('Classification Report:')
print(classification_report(y_test, y1_pred))

f1_macro = f1_score(y_test, y1_pred, average='macro')

print(f'Macro-average F1 score: {f1_macro:.4f}')

Accuracy: 0.8552
Confusion Matrix:
[[6443  202]
 [ 934  264]]
Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.97      0.92      6645
           1       0.57      0.22      0.32      1198

    accuracy                           0.86      7843
   macro avg       0.72      0.59      0.62      7843
weighted avg       0.83      0.86      0.83      7843

Macro-average F1 score: 0.6181


**Logistic regression result**
* **Confusion Matrix:** The model has more true negatives (class 0 correctly predicted) than true positives (class 1 correctly predicted). There are many false positives and false negatives, which might indicate class imbalance or issues in model performance for class 1.
* **Classification Report:**
  * The model performs well on class 0 (high precision, recall, and F1-score).
  * The model performs poorly on class 1 (low precision, recall, and F1-score).
  * Macro-average F1-score: 0.6181, indicating a poor balance between precision and recall.


In [43]:

# pred = model1.predict(test_pre)
# original_predictions = label_encoder.inverse_transform(pred)

# submission = pd.DataFrame({"id":range(0,test_pre.shape[0]),'target':original_predictions})
# submission.to_csv("submission.csv",index = False)

**LogisticRegression Hyperparameter Tune**

In [44]:
#hyper parameter tuning for LogisticRegression using GridSearchCV

model11 = LogisticRegression(random_state=42)


param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],  # Regularization strength.Smaller values indicate stronger regularization.
    'penalty': ['l2'],  # L2 penalty 
    'solver': ['liblinear', 'lbfgs'],  # solvers algorithm used to optimize the logistic regression model.
    'max_iter': [100, 200, 500],  # maximum number of iterations for the optimization algorithm
}

# hyperparameter search
grid_search11 = GridSearchCV(estimator=model11, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)

# Fitting training data
grid_search11.fit(X_train, y_train)


best_params = grid_search11.best_params_
best_score = grid_search11.best_score_

print(f"Best Hyperparameters: {best_params}")
print(f"Best Cross-validation Accuracy: {best_score:.4f}")



Fitting 5 folds for each of 30 candidates, totalling 150 fits
Best Hyperparameters: {'C': 1, 'max_iter': 100, 'penalty': 'l2', 'solver': 'liblinear'}
Best Cross-validation Accuracy: 0.8595


In [45]:

best_model = grid_search11.best_estimator_
y11_pred = best_model.predict(X_test)

accuracy_best = accuracy_score(y_test, y11_pred)
print(f'Accuracy: {accuracy_best:.4f}')
print('Confusion Matrix:')
print(confusion_matrix(y_test, y11_pred))
print('Classification Report:')
print(classification_report(y_test, y11_pred))

f1_macro = f1_score(y_test, y11_pred, average='macro')

print(f'Macro-average F1 score: {f1_macro:.4f}')



Accuracy: 0.8552
Confusion Matrix:
[[6443  202]
 [ 934  264]]
Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.97      0.92      6645
           1       0.57      0.22      0.32      1198

    accuracy                           0.86      7843
   macro avg       0.72      0.59      0.62      7843
weighted avg       0.83      0.86      0.83      7843

Macro-average F1 score: 0.6181


after hyper parameter tuning no improvement on score .
* relationships between features and the target are non-linear.
* have a class imbalance.

so i try class_weight='balanced' for logisticRegression.


In [46]:
#LogisticRegression with class_weight='balanced'

model12 = LogisticRegression(max_iter=1000, random_state=42, class_weight='balanced')

model12.fit(X_train, y_train)

y12_pred = model12.predict(X_test)


accuracy = accuracy_score(y_test, y12_pred)
print(f'Accuracy: {accuracy:.4f}')
print('Confusion Matrix:')
print(confusion_matrix(y_test, y12_pred))
print('Classification Report:')
print(classification_report(y_test, y12_pred))

f1_macro = f1_score(y_test, y12_pred, average='macro')
print(f'Macro-average F1 score: {f1_macro:.4f}')


Accuracy: 0.8221
Confusion Matrix:
[[5655  990]
 [ 405  793]]
Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.85      0.89      6645
           1       0.44      0.66      0.53      1198

    accuracy                           0.82      7843
   macro avg       0.69      0.76      0.71      7843
weighted avg       0.86      0.82      0.84      7843

Macro-average F1 score: 0.7111


* Accuracy: 0.8221 , Decreases, but more balanced.
* F1 Score (Macro): 0.7111.
* The recall for class 1 increased from 0.22 to 0.66, which is a huge improvement.
* f1-score Improves for the minority class

here this model12 shows good balance between precision and recall for the positive class (1), but the recall for the negative class (0) was quite high, indicating potential over-prediction of the negative class.

In [47]:
pipeline_lr = Pipeline([
    ('logreg', LogisticRegression(class_weight='balanced', random_state=42)) 
])

param_grid_lr = {
    'logreg__C': [0.01, 0.1, 1, 10, 100],  # Regularization strength
    'logreg__penalty': ['l2'],  # L2 penalty   to avoid overfitting
    'logreg__solver': ['liblinear', 'lbfgs'],  # Solvers
    'logreg__max_iter': [100, 200, 500],  # Max iterations
}


grid_search_lr = GridSearchCV(estimator=pipeline_lr, param_grid=param_grid_lr, cv=5, scoring='f1_macro', verbose=1, n_jobs=-1)


grid_search_lr.fit(X_train, y_train)


best_params_lr = grid_search_lr.best_params_
best_score_lr = grid_search_lr.best_score_

print(f"Best Hyperparameters: {best_params_lr}")
print(f"Best Cross-validation Accuracy: {best_score_lr:.4f}")


best_model_lr = grid_search_lr.best_estimator_


y13_pred = best_model_lr.predict(X_test)


accuracy_best_lr = accuracy_score(y_test, y13_pred)
print(f'Accuracy: {accuracy_best_lr:.4f}')
print('Confusion Matrix:')
print(confusion_matrix(y_test, y13_pred))
print('Classification Report:')
print(classification_report(y_test, y13_pred))

# Calculate F1 score (macro)
f1_macro_best_lr = f1_score(y_test, y13_pred, average='macro')
print(f'Macro-average F1 score: {f1_macro_best_lr:.4f}')

Fitting 5 folds for each of 30 candidates, totalling 150 fits
Best Hyperparameters: {'logreg__C': 0.01, 'logreg__max_iter': 100, 'logreg__penalty': 'l2', 'logreg__solver': 'liblinear'}
Best Cross-validation Accuracy: 0.7179
Accuracy: 0.8237
Confusion Matrix:
[[5663  982]
 [ 401  797]]
Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.85      0.89      6645
           1       0.45      0.67      0.54      1198

    accuracy                           0.82      7843
   macro avg       0.69      0.76      0.71      7843
weighted avg       0.86      0.82      0.84      7843

Macro-average F1 score: 0.7133


* Accuracy: 0.8237 (slightly improved from the initial model) improved (82.21% to 82.37%)
* Recall for class 1 improved (0.66 to 0.67)
* Macro-average F1 score: 0.7133 ( better than before also clear the cutoff) improved (0.7111 to 0.7133)
*  negative class (0), with a high precision and recall for it.


In [48]:
pred = best_model_lr.predict(test_pre)
original_predictions = label_encoder.inverse_transform(pred)

submission = pd.DataFrame({"id":range(0,test_pre.shape[0]),'target':original_predictions})
submission.to_csv("submission.csv",index = False)

 increasing the C range (like 0.001, 0.005)

In [49]:
pipeline_lr2 = Pipeline([
    ('logreg', LogisticRegression(class_weight='balanced', random_state=42)) 
])

param_distributions_lr = {
    'logreg__C': [0.001, 0.005, 0.1, 10, 100],  # Regularization strength
    'logreg__penalty': ['l2'],  # L2 penalty
    'logreg__solver': ['liblinear', 'lbfgs'],  # Solvers
    'logreg__max_iter': [100, 200, 500],  # Max iterations
}


random_search_lr = RandomizedSearchCV(
    estimator=pipeline_lr,
    param_distributions=param_distributions_lr,
    n_iter=20,  # Number of random parameter combinations
    cv=5,  
    scoring='f1_macro',
    verbose=1,
    random_state=42,#displays a progress bar with training and validation metrics
    n_jobs=-1
)


random_search_lr.fit(X_train, y_train)


best_params_lr = random_search_lr.best_params_
best_score_lr = random_search_lr.best_score_

print(f"Best Hyperparameters: {best_params_lr}")
print(f"Best Cross-validation Accuracy: {best_score_lr:.4f}")


best_model_lr2 = random_search_lr.best_estimator_
y14_pred = best_model_lr2.predict(X_test)

accuracy_best_lr = accuracy_score(y_test, y14_pred)
print(f'Accuracy: {accuracy_best_lr:.4f}')
print('Confusion Matrix:')
print(confusion_matrix(y_test, y14_pred))
print('Classification Report:')
print(classification_report(y_test, y14_pred))

# Calculate F1 score (macro)
f1_macro_best_lr = f1_score(y_test, y14_pred, average='macro')
print(f'Macro-average F1 score: {f1_macro_best_lr:.4f}')

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best Hyperparameters: {'logreg__solver': 'liblinear', 'logreg__penalty': 'l2', 'logreg__max_iter': 100, 'logreg__C': 0.001}
Best Cross-validation Accuracy: 0.7218
Accuracy: 0.8307
Confusion Matrix:
[[5753  892]
 [ 436  762]]
Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.87      0.90      6645
           1       0.46      0.64      0.53      1198

    accuracy                           0.83      7843
   macro avg       0.70      0.75      0.72      7843
weighted avg       0.86      0.83      0.84      7843

Macro-average F1 score: 0.7154


* Macro-average F1 score:  0.7154,  improved 

both methods found the same optimal hyperparameters and resulted in nearly identical performance metrics
RandomizedSearchCV is the preferred choice here due to its computational efficiency 
(100 fits vs. 150 fits).
 it requires less time and resources to search for the optimal hyperparameters

In [50]:
pred = best_model_lr2.predict(test_pre)
original_predictions = label_encoder.inverse_transform(pred)

submission = pd.DataFrame({"id":range(0,test_pre.shape[0]),'target':original_predictions})
submission.to_csv("submission.csv",index = False)

**Perceptron**

In [51]:
model_perceptron = Perceptron(random_state=42, class_weight='balanced')

model_perceptron.fit(X_train, y_train)

y_pred_perceptron = model_perceptron.predict(X_test)


accuracy_perceptron = accuracy_score(y_test, y_pred_perceptron)
print(f'Accuracy: {accuracy_perceptron:.4f}')


print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred_perceptron))


print('Classification Report:')
print(classification_report(y_test, y_pred_perceptron))


f1_macro_perceptron = f1_score(y_test, y_pred_perceptron, average='macro')
print(f'Macro-average F1 score: {f1_macro_perceptron:.4f}')

Accuracy: 0.6209
Confusion Matrix:
[[4044 2601]
 [ 372  826]]
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.61      0.73      6645
           1       0.24      0.69      0.36      1198

    accuracy                           0.62      7843
   macro avg       0.58      0.65      0.54      7843
weighted avg       0.81      0.62      0.67      7843

Macro-average F1 score: 0.5442


* Accuracy (62.09%): Relatively low compared to previous models like Logistic Regression (which achieved 83%). This suggests that the model may be struggling to generalize.
* Precision for Class 1 (0.24): Indicates that only 24% of the predicted "1s" are correct. There are many false positives.
*  identifying the majority class (0) but struggles with the minority class (1).
* for 0 ,F1-score: 73% - Balanced measure of precision and recall.
* for 1, F1-score: 36% - Indicates the model is weak in handling the minority class.

  
Perceptron is a linear classifier and is sensitive to imbalanced datasets, making it less capable of capturing complex decision boundaries or minority class patterns in imbalanced data.

In [52]:
# #hyper parameter tuning for Perceptron using RandomizedSearchCV
pipeline_perceptron = Pipeline([
    ('perceptron', Perceptron(random_state=42, class_weight='balanced'))
])
param_grid_perceptron = {
    'perceptron__penalty': [None, 'l2', 'l1', 'elasticnet'],  # Regularization
    'perceptron__alpha': [0.0001, 0.001, 0.01, 0.1],         # Learning rate
    'perceptron__max_iter': [100, 500, 1000],                # Maximum number of iterations
    'perceptron__eta0': [0.01, 0.1, 1],                     # Initial learning rate
    'perceptron__fit_intercept': [True, False],              # Whether to fit the intercept
}

random_search_perceptron = RandomizedSearchCV(
    estimator=pipeline_perceptron,
    param_distributions=param_grid_perceptron,
    n_iter=20,  # Number of random samples
    cv=5,
    scoring='accuracy',
    verbose=1,
    n_jobs=-1,
    random_state=42
)


random_search_perceptron.fit(X_train, y_train)


best_params_random_perceptron = random_search_perceptron.best_params_
best_score_random_perceptron = random_search_perceptron.best_score_

print(f"Best Hyperparameters: {best_params_random_perceptron}")
print(f"Best Cross-validation Accuracy: {best_score_random_perceptron:.4f}")


best_model_random_perceptron = random_search_perceptron.best_estimator_
y_pred_random_perceptron = best_model_random_perceptron.predict(X_test)


accuracy_random_perceptron = accuracy_score(y_test, y_pred_random_perceptron)
print(f'Accuracy: {accuracy_random_perceptron:.4f}')
print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred_random_perceptron))
print('Classification Report:')
print(classification_report(y_test, y_pred_random_perceptron))


f1_macro_random_perceptron = f1_score(y_test, y_pred_random_perceptron, average='macro')
print(f'Macro-average F1 score: {f1_macro_random_perceptron:.4f}')




Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best Hyperparameters: {'perceptron__penalty': 'l2', 'perceptron__max_iter': 500, 'perceptron__fit_intercept': True, 'perceptron__eta0': 0.01, 'perceptron__alpha': 0.001}
Best Cross-validation Accuracy: 0.8051
Accuracy: 0.7037
Confusion Matrix:
[[4850 1795]
 [ 529  669]]
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.73      0.81      6645
           1       0.27      0.56      0.37      1198

    accuracy                           0.70      7843
   macro avg       0.59      0.64      0.59      7843
weighted avg       0.81      0.70      0.74      7843

Macro-average F1 score: 0.5860


* accuracy improved to 70.37% from the untuned model's 62.09%, reflecting a more balanced classification ability overall.
* The minority class (1) has significantly fewer samples, leading to poor performance on precision and recall for that class.
* The Perceptron is a linear model and may not capture complex patterns in the data.
  

In [53]:
# Ridge Classifier
ridge_clf = RidgeClassifier(class_weight='balanced', random_state=42)


ridge_clf.fit(X_train, y_train)


y_ridge_pred = ridge_clf.predict(X_test)


accuracy_ridge = accuracy_score(y_test, y_ridge_pred)
conf_matrix_ridge = confusion_matrix(y_test, y_ridge_pred)
class_report_ridge = classification_report(y_test, y_ridge_pred)
f1_macro_ridge = f1_score(y_test, y_ridge_pred, average='macro')

print(f"Accuracy: {accuracy_ridge:.4f}")
print("Confusion Matrix:")
print(conf_matrix_ridge)
print("Classification Report:")
print(class_report_ridge)
print(f"Macro-average F1 score: {f1_macro_ridge:.4f}")


Accuracy: 0.8058
Confusion Matrix:
[[5598 1047]
 [ 476  722]]
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.84      0.88      6645
           1       0.41      0.60      0.49      1198

    accuracy                           0.81      7843
   macro avg       0.66      0.72      0.68      7843
weighted avg       0.84      0.81      0.82      7843

Macro-average F1 score: 0.6835


* Precision (for Class 0): 0.92, which indicates that the model is very accurate when predicting the negative class (0).
* Recall (for Class 0): 0.84, meaning the model correctly identifies 84% of the negative class (0).
* Precision (for Class 1): 0.41, indicating that when the model predicts the positive class (1), it's not very precise.
*  Recall (for Class 1): 0.60, meaning the model is able to correctly identify 60% of the positive class (1).
*  The class imbalance is evident from the performance: while the model does well at predicting class 0 (precision 92%, recall 84%), it struggles more with class 1 (precision 41%, recall 60%).

 The F1 score of 0.6835 is a reasonable metric, but there's still need to improve, especially for the positive class (1).

In [54]:
#hyperparameter tune of RidgeClassifier

model_ridge = RidgeClassifier(class_weight='balanced')


param_dist_ridge = {
    'alpha': np.logspace(-6, 6, 50),  # Range for alpha (regularization strength)Higher alpha stronger regularization
    'fit_intercept': [True, False],  # Whether to include an intercept in the model
}


random_search_ridge = RandomizedSearchCV(
    estimator=model_ridge, 
    param_distributions=param_dist_ridge, 
    n_iter=100,  # Number of parameter settings to sample
    cv=5,  # 5-fold cross-validation
    verbose=1, 
    n_jobs=-1,  # Use all available cores
    scoring='f1_macro'
)


random_search_ridge.fit(X_train, y_train)


best_params_ridge = random_search_ridge.best_params_
best_score_ridge = random_search_ridge.best_score_

print(f"Best Hyperparameters for Ridge Classifier: {best_params_ridge}")
print(f"Best Cross-validation Accuracy: {best_score_ridge:.4f}")


best_model_ridge = random_search_ridge.best_estimator_
y_ridge_pred = best_model_ridge.predict(X_test)


accuracy_ridge = accuracy_score(y_test, y_ridge_pred)
print(f'Accuracy: {accuracy_ridge:.4f}')
print('Confusion Matrix:')
print(confusion_matrix(y_test, y_ridge_pred))
print('Classification Report:')
print(classification_report(y_test, y_ridge_pred))


f1_macro_ridge = f1_score(y_test, y_ridge_pred, average='macro')
print(f'Macro-average F1 score: {f1_macro_ridge:.4f}')


Fitting 5 folds for each of 100 candidates, totalling 500 fits
Best Hyperparameters for Ridge Classifier: {'fit_intercept': False, 'alpha': 19306.977288832455}
Best Cross-validation Accuracy: 0.7172
Accuracy: 0.8442
Confusion Matrix:
[[5995  650]
 [ 572  626]]
Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.90      0.91      6645
           1       0.49      0.52      0.51      1198

    accuracy                           0.84      7843
   macro avg       0.70      0.71      0.71      7843
weighted avg       0.85      0.84      0.85      7843

Macro-average F1 score: 0.7068


* The hyperparameter tuning using RandomizedSearchCV has improved the Ridge Classifier's performance compared to the default model.
* The best regularization strength (alpha) is around 19306.98, and the model performs better without an intercept.
* The accuracy of 84.42% on the test set is a significant improvement, but the model still has some challenges with predicting class 1.
* 

In [55]:
pred = best_model_ridge.predict(test_pre)
original_predictions = label_encoder.inverse_transform(pred)

submission = pd.DataFrame({"id":range(0,test_pre.shape[0]),'target':original_predictions})
submission.to_csv("submission.csv",index = False)

In [56]:
# #  KNN classifier 
model_knn = KNeighborsClassifier()


model_knn.fit(X_train, y_train)


y_knn_pred = model_knn.predict(X_test)


accuracy_knn = accuracy_score(y_test, y_knn_pred)
print(f'Accuracy: {accuracy_knn:.4f}')


print('Confusion Matrix:')
print(confusion_matrix(y_test, y_knn_pred))


print('Classification Report:')
print(classification_report(y_test, y_knn_pred))


f1_macro_knn = f1_score(y_test, y_knn_pred, average='macro')
print(f'Macro-average F1 score: {f1_macro_knn:.4f}')


Accuracy: 0.8552
Confusion Matrix:
[[6374  271]
 [ 865  333]]
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.96      0.92      6645
           1       0.55      0.28      0.37      1198

    accuracy                           0.86      7843
   macro avg       0.72      0.62      0.64      7843
weighted avg       0.83      0.86      0.83      7843

Macro-average F1 score: 0.6439


Accuracy: 0.8552* The KNN classifier has good performance on the majority class (class 0), with high precision, recall, and F1-score. However, it struggles with the minority class (class 1), where it has lower precision, recall, and F1-score.
* The macro-average F1 score is relatively low score: 0.6439

In [57]:
# #tune knn classifier

# model_knn = KNeighborsClassifier()


# param_dist_knn = {
#     'n_neighbors': np.arange(1, 21),  # Number of neighbors
#     'weights': ['uniform', 'distance'],  # Weighting method
#     'metric': ['euclidean', 'manhattan', 'minkowski'],  # Distance metrics
#     'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],  # Search algorithm
# }


# random_search_knn = RandomizedSearchCV(
#     estimator=model_knn,
#     param_distributions=param_dist_knn,
#     n_iter=50,  
#     cv=5,  # 5-fold cross-validation
#     verbose=1,
#     n_jobs=-1,  # Use all available cores
#     scoring='f1_macro',
#     random_state=42  
# )


# random_search_knn.fit(X_train, y_train)


# best_params_knn = random_search_knn.best_params_
# best_score_knn = random_search_knn.best_score_

# print(f"Best Hyperparameters for KNN: {best_params_knn}")
# print(f"Best Cross-validation Accuracy: {best_score_knn:.4f}")


# best_model_knn = random_search_knn.best_estimator_
# y_knn_pred = best_model_knn.predict(X_test)


# accuracy_knn = accuracy_score(y_test, y_knn_pred)
# print(f'Accuracy: {accuracy_knn:.4f}')
# print('Confusion Matrix:')
# print(confusion_matrix(y_test, y_knn_pred))
# print('Classification Report:')
# print(classification_report(y_test, y_knn_pred))

# f1_macro_knn = f1_score(y_test, y_knn_pred, average='macro')
# print(f'Macro-average F1 score: {f1_macro_knn:.4f}')


Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best Hyperparameters for KNN: {'weights': 'distance', 'n_neighbors': 1, 'metric': 'manhattan', 'algorithm': 'kd_tree'}
Best Cross-validation Accuracy: 0.6514
Accuracy: 0.8344
Confusion Matrix:
[[6135  510]
 [ 789  409]]
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.92      0.90      6645
           1       0.45      0.34      0.39      1198

    accuracy                           0.83      7843
   macro avg       0.67      0.63      0.65      7843
weighted avg       0.82      0.83      0.83      7843

Macro-average F1 score: 0.6453

* The KNN model performed well in terms of accuracy on the test set (83.44%), but it struggled with class 1 (positive class) as indicated by the low recall and precision for this class.
* The best hyperparameters found through random search are:
        n_neighbors=1
        weights='distance'
        metric='manhattan'
        algorithm='kd_tree'
* The model's performance is biased towards the majority class (class 0), which is common in imbalanced datasets. The macro-average F1 score (0.6453) reflects this imbalance

In [58]:
# # SGDClassifier
model_sgd = SGDClassifier(random_state=42 )


model_sgd.fit(X_train, y_train)


y_sgd_pred = model_sgd.predict(X_test)


accuracy_sgd = accuracy_score(y_test, y_sgd_pred)
print(f'Accuracy: {accuracy_sgd:.4f}')


print('Confusion Matrix:')
print(confusion_matrix(y_test, y_sgd_pred))


print('Classification Report:')
print(classification_report(y_test, y_sgd_pred))


f1_macro_sgd = f1_score(y_test, y_sgd_pred, average='macro')
print(f'Macro-average F1 score: {f1_macro_sgd:.4f}')


Accuracy: 0.8562
Confusion Matrix:
[[6385  260]
 [ 868  330]]
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.96      0.92      6645
           1       0.56      0.28      0.37      1198

    accuracy                           0.86      7843
   macro avg       0.72      0.62      0.64      7843
weighted avg       0.83      0.86      0.83      7843

Macro-average F1 score: 0.6440


* The model performs well on class 0 (negative class), with good precision (0.88) and recall (0.96). This indicates that the model correctly identifies most instances of the negative class.
* model struggles with class 1 (positive class), as evidenced by low recall (0.28) and precision (0.56). This suggests that the model is not good at identifying the positive class, possibly due to class imbalance.
* he macro-average F1 score (0.6440) indicates moderate overall performance, but it shows that there’s room for improvement, especially for the minority class.

In [59]:
# # SGDClassifier weight balanced
model_sgd = SGDClassifier(class_weight='balanced',random_state=42)


model_sgd.fit(X_train, y_train)


y_sgd_pred = model_sgd.predict(X_test)


accuracy_sgd = accuracy_score(y_test, y_sgd_pred)
print(f'Accuracy: {accuracy_sgd:.4f}')


print('Confusion Matrix:')
print(confusion_matrix(y_test, y_sgd_pred))


print('Classification Report:')
print(classification_report(y_test, y_sgd_pred))


f1_macro_sgd = f1_score(y_test, y_sgd_pred, average='macro')
print(f'Macro-average F1 score: {f1_macro_sgd:.4f}')

Accuracy: 0.8181
Confusion Matrix:
[[5592 1053]
 [ 374  824]]
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.84      0.89      6645
           1       0.44      0.69      0.54      1198

    accuracy                           0.82      7843
   macro avg       0.69      0.76      0.71      7843
weighted avg       0.86      0.82      0.83      7843

Macro-average F1 score: 0.7114


*  using class_weight='balanced', the model improved recall for class 1, but it came at the cost of a lower precision for class 1.
*  The macro-average F1 score of 0.7114 suggests that while the model performs fairly well overall, there's still potential for further refinement, especially for class 1.

In [60]:
# # SGDClassifier tuned
model_sgd = SGDClassifier(class_weight='balanced')


param_dist_sgd = {
    'loss': ['hinge', 'log_loss', 'squared_hinge'],  # Loss functions
    'penalty': ['l2', 'l1', 'elasticnet'],  # Regularization techniques
    'alpha': np.logspace(-6, 6, 13),  # Regularization strength
    'max_iter': [1000, 2000, 3000, 5000],  # Number of iterations
    'learning_rate': ['constant', 'optimal', 'invscaling'],  # Learning rate schedules
}


random_search_sgd = RandomizedSearchCV(
    estimator=model_sgd,
    param_distributions=param_dist_sgd,
    n_iter=50,  # Number of parameter settings to sample
    cv=5,  # 5-fold cross-validation
    verbose=1,
    n_jobs=-1,  # Use all available cores
    scoring='f1_macro',
    random_state=42
    
)


random_search_sgd.fit(X_train, y_train)


best_params_sgd = random_search_sgd.best_params_
best_score_sgd = random_search_sgd.best_score_

print(f"Best Hyperparameters for SGDClassifier: {best_params_sgd}")
print(f"Best Cross-validation Accuracy: {best_score_sgd:.4f}")


best_model_sgd = random_search_sgd.best_estimator_


y_sgd_pred = best_model_sgd.predict(X_test)


accuracy_sgd = accuracy_score(y_test, y_sgd_pred)
print(f'Accuracy: {accuracy_sgd:.4f}')
print('Confusion Matrix:')
print(confusion_matrix(y_test, y_sgd_pred))
print('Classification Report:')
print(classification_report(y_test, y_sgd_pred))


f1_macro_sgd = f1_score(y_test, y_sgd_pred, average='macro')
print(f'Macro-average F1 score: {f1_macro_sgd:.4f}')

Fitting 5 folds for each of 50 candidates, totalling 250 fits


/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
175 fits failed out of a total of 250.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
175 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py", line 892, in fit
    self._more_validate_params()
  File "/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py", line 149, in _more_validate_params
    raise ValueError("eta0 must be > 0

Best Hyperparameters for SGDClassifier: {'penalty': 'l1', 'max_iter': 3000, 'loss': 'hinge', 'learning_rate': 'optimal', 'alpha': 0.001}
Best Cross-validation Accuracy: 0.7414
Accuracy: 0.8447
Confusion Matrix:
[[5815  830]
 [ 388  810]]
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.88      0.91      6645
           1       0.49      0.68      0.57      1198

    accuracy                           0.84      7843
   macro avg       0.72      0.78      0.74      7843
weighted avg       0.87      0.84      0.85      7843

Macro-average F1 score: 0.7380


Best Hyperparameters for SGDClassifier: {'penalty': 'l1', 'max_iter': 3000, 'loss': 'hinge', 'learning_rate': 'optimal', 'alpha': 0.001}
Best Cross-validation Accuracy: 0.7420
Accuracy: 0.8437
Confusion Matrix:
[[5820  825]
 [ 401  797]]
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.88      0.90      6645
           1       0.49      0.67      0.57      1198

    accuracy                           0.84      7843
   macro avg       0.71      0.77      0.73      7843
weighted avg       0.87      0.84      0.85      7843

Macro-average F1 score: 0.7350

* The model's performance has improved after hyperparameter tuning, with better accuracy, recall for class 1, and a higher macro-average F1 score.
* class 1 predictions are still weaker (precision for class 1 is low). However, the model is more balanced than before.
*  hyperparameter tuning process helped optimize the model’s configuration to achieve these improvements.

In [61]:
pred = best_model_sgd.predict(test_pre)
original_predictions = label_encoder.inverse_transform(pred)

submission = pd.DataFrame({"id":range(0,test_pre.shape[0]),'target':original_predictions})
submission.to_csv("submission.csv",index = False)

In [62]:
# #  SVM model
model_svm = SVC(class_weight='balanced',random_state=42)


model_svm.fit(X_train, y_train)


y_svm_pred = model_svm.predict(X_test)


accuracy_svm = accuracy_score(y_test, y_svm_pred)
print(f'Accuracy: {accuracy_svm:.4f}')
print('Confusion Matrix:')
print(confusion_matrix(y_test, y_svm_pred))
print('Classification Report:')
print(classification_report(y_test, y_svm_pred))


f1_macro_svm = f1_score(y_test, y_svm_pred, average='macro')
print(f'Macro-average F1 score: {f1_macro_svm:.4f}')


Accuracy: 0.8210
Confusion Matrix:
[[5463 1182]
 [ 222  976]]
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.82      0.89      6645
           1       0.45      0.81      0.58      1198

    accuracy                           0.82      7843
   macro avg       0.71      0.82      0.73      7843
weighted avg       0.88      0.82      0.84      7843

Macro-average F1 score: 0.7339


* The SVM model provides a solid performance with 82.10% accuracy and a macro-average F1 score of 0.73.
* The model's recall for class 1 is 81%, indicating it correctly identifies many instances of the minority class. However, the precision for class 1 is low (45%), meaning there are many false positives.
* Class 0 is predicted with high precision (96%), but recall for class 0 is 82%, meaning that some class 0 instances are missed.
* Compared to the SGDClassifier model, the SVM model has a lower overall accuracy but a higher recall for the minority class (class 1), which could be important depending on the specific application.

In [63]:
pred = model_svm.predict(test_pre)
original_predictions = label_encoder.inverse_transform(pred)

submission = pd.DataFrame({"id":range(0,test_pre.shape[0]),'target':original_predictions})
submission.to_csv("submission.csv",index = False)

In [64]:
# import warnings
# warnings.filterwarnings("ignore")

# # SVM model parameter tuning 
# model_svm = SVC(C=10,class_weight='balanced')

# param_dist_svm_refined = {
#     'C': [0.1, 1, 10],  # Narrowed down the regularization parameter based on previous results
#     'kernel': ['rbf'],  # Focusing on 'rbf' and 'linear' kernels for more efficient tuning
#     'gamma': ['scale'],  # Common gamma values
#     'degree': [3],  # Default value for degree
#     'max_iter': [1000],  # Max iterations for SVC (kept constant)
# }

# random_search_svm = RandomizedSearchCV(
#     estimator=model_svm,
#     param_distributions=param_dist_svm_refined,
#     n_iter=10,  # Number of parameter settings that are sampled
#     cv=5,  # Cross-validation splitting strategy
#     verbose=1,
#     random_state=42,
#     n_jobs=-1,  # Using all available CPU cores
#     scoring='f1_macro'  # Using F1 macro score to evaluate performance
# )

# random_search_svm.fit(X_train, y_train)



# best_params_svm = random_search_svm.best_params_
# best_score_svm = random_search_svm.best_score_


# print(f"Best Hyperparameters for SVM: {best_params_svm}")
# print(f"Best Cross-validation Accuracy: {best_score_svm:.4f}")


# best_model_svm = random_search_svm.best_estimator_

# y_svm_pred_tuned = best_model_svm.predict(X_test)


# accuracy_svm_tuned = accuracy_score(y_test, y_svm_pred_tuned)
# print(f'Accuracy: {accuracy_svm_tuned:.4f}')
# print('Confusion Matrix:')
# print(confusion_matrix(y_test, y_svm_pred_tuned))
# print('Classification Report:')
# print(classification_report(y_test, y_svm_pred_tuned))


# f1_macro_svm_tuned = f1_score(y_test, y_svm_pred_tuned, average='macro')
# print(f'Macro-average F1 score: {f1_macro_svm_tuned:.4f}')


Macro-average F1 score: 0.3000  

In [65]:
model_rf = RandomForestClassifier(class_weight='balanced',random_state=42, n_jobs=-1)


model_rf.fit(X_train, y_train)


y_rf_pred = model_rf.predict(X_test)

 
accuracy_rf = accuracy_score(y_test, y_rf_pred)
print(f'Accuracy: {accuracy_rf:.4f}')


print('Confusion Matrix:')
print(confusion_matrix(y_test, y_rf_pred))


print('Classification Report:')
print(classification_report(y_test, y_rf_pred))


f1_macro_rf = f1_score(y_test, y_rf_pred, average='macro')
print(f'Macro-average F1 score: {f1_macro_rf:.4f}')

Accuracy: 0.8628
Confusion Matrix:
[[6402  243]
 [ 833  365]]
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.96      0.92      6645
           1       0.60      0.30      0.40      1198

    accuracy                           0.86      7843
   macro avg       0.74      0.63      0.66      7843
weighted avg       0.84      0.86      0.84      7843

Macro-average F1 score: 0.6633


In [66]:
model_rf = RandomForestClassifier(class_weight='balanced',random_state=42)


param_dist_rf = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2'],
    'bootstrap': [True, False]
}


random_search_rf = RandomizedSearchCV(
    estimator=model_rf,
    param_distributions=param_dist_rf,
    n_iter=10,  # Number of random parameter
    cv=5,  
    verbose=1,
    random_state=42,
    n_jobs=-1,  
    scoring='f1_macro'  
)


random_search_rf.fit(X_train, y_train)


best_params_rf = random_search_rf.best_params_
best_score_rf = random_search_rf.best_score_

print(f"Best Hyperparameters for Random Forest: {best_params_rf}")
print(f"Best Cross-validation F1 Macro Score: {best_score_rf:.4f}")


best_model_rf = random_search_rf.best_estimator_


y_rf_pred_tuned = best_model_rf.predict(X_test)


accuracy_rf_tuned = accuracy_score(y_test, y_rf_pred_tuned)
print(f'Accuracy: {accuracy_rf_tuned:.4f}')


print('Confusion Matrix:')
print(confusion_matrix(y_test, y_rf_pred_tuned))


print('Classification Report:')
print(classification_report(y_test, y_rf_pred_tuned))


f1_macro_rf_tuned = f1_score(y_test, y_rf_pred_tuned, average='macro')
print(f'Macro-average F1 score: {f1_macro_rf_tuned:.4f}')

Fitting 5 folds for each of 10 candidates, totalling 50 fits


/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomFor

Best Hyperparameters for Random Forest: {'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': 'log2', 'max_depth': None, 'bootstrap': True}
Best Cross-validation F1 Macro Score: 0.7661
Accuracy: 0.8578
Confusion Matrix:
[[5799  846]
 [ 269  929]]
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.87      0.91      6645
           1       0.52      0.78      0.62      1198

    accuracy                           0.86      7843
   macro avg       0.74      0.82      0.77      7843
weighted avg       0.89      0.86      0.87      7843

Macro-average F1 score: 0.7686


Best Hyperparameters for Random Forest: {'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': 'log2', 'max_depth': None, 'bootstrap': True}
Best Cross-validation F1 Macro Score: 0.7661
Accuracy: 0.8578
Confusion Matrix:
[[5799  846]
 [ 269  929]]
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.87      0.91      6645
           1       0.52      0.78      0.62      1198

    accuracy                           0.86      7843
   macro avg       0.74      0.82      0.77      7843
weighted avg       0.89      0.86      0.87      7843

Macro-average F1 score: 0.7686

* The Random Forest Classifier after tuning shows strong performance with good accuracy (85.78%) and a good balance between precision and recall. The macro F1 score (0.7686) indicates a fair overall performance across both classes, with some room for improvement in the precision for the minority class (class 1).
* Class 0 (negative class):
    Precision: 0.96, Recall: 0.87, F1-Score: 0.91
    Class 0 is classified very well with high precision and recall.
* Class 1 (positive class):
    Precision: 0.52, Recall: 0.78, F1-Score: 0.62
    Class 1 has a lower precision, but the recall is relatively good. This suggests the model is able to correctly identify a significant number of positive cases, but there are a number of false positives.

this model may be overfitting.

In [67]:
pred = best_model_rf.predict(test_pre)
original_predictions = label_encoder.inverse_transform(pred)

submission = pd.DataFrame({"id":range(0,test_pre.shape[0]),'target':original_predictions})
submission.to_csv("submission.csv",index = False)

In [68]:
#XGBoost Classifier

xgb_default = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_default.fit(X_train, y_train)


y_pred_xgb_default = xgb_default.predict(X_test)

# Evaluation
accuracy_xgb_default = accuracy_score(y_test, y_pred_xgb_default)
f1_macro_xgb_default = f1_score(y_test, y_pred_xgb_default, average='macro')
conf_matrix_xgb_default = confusion_matrix(y_test, y_pred_xgb_default)
class_report_xgb_default = classification_report(y_test, y_pred_xgb_default)

print("Default XGBoost Classifier:")
print(f"Accuracy: {accuracy_xgb_default:.4f}")
print(f"Macro F1 Score: {f1_macro_xgb_default:.4f}")
print("Confusion Matrix:")
print(conf_matrix_xgb_default)
print("Classification Report:")
print(class_report_xgb_default)

Default XGBoost Classifier:
Accuracy: 0.8692
Macro F1 Score: 0.7211
Confusion Matrix:
[[6266  379]
 [ 647  551]]
Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.94      0.92      6645
           1       0.59      0.46      0.52      1198

    accuracy                           0.87      7843
   macro avg       0.75      0.70      0.72      7843
weighted avg       0.86      0.87      0.86      7843



* Precision for class 0 is high (0.91), meaning when the model predicts class 0, it's often correct.
* Recall for class 1 is relatively low (0.46), indicating that the model is missing many instances of class 1.
* Macro F1 Score: 0.7211  means the model performs reasonably well across both classes

In [69]:
# #XGBoost Classifier tuning
# param_dist_xgb = {
#     'n_estimators': [100, 200, 300],  # Number of trees
#     'learning_rate': [0.01, 0.05, 0.1, 0.2],  # Step size shrinkage
#     'max_depth': [3, 5, 7, 10],  # Maximum tree depth
#     'min_child_weight': [1, 3, 5],  # Minimum sum of instance weight
#     'gamma': [0, 0.1, 0.3, 0.5],  # Minimum loss reduction
#     'subsample': [0.6, 0.8, 1.0],  # Subsampling ratio
#     'colsample_bytree': [0.6, 0.8, 1.0],  # Fraction of features for tree
#     'scale_pos_weight': [1, 5, 10],  # Balancing positive and negative classes
# }


# xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

# random_search_xgb = RandomizedSearchCV(
#     estimator=xgb,
#     param_distributions=param_grid_xgb,
#     n_iter=20,
#     cv=5,
#     scoring='f1_macro',
#     verbose=2,
#     random_state=42,
#     n_jobs=-1
# )


# random_search_xgb.fit(X_train, y_train)

# best_params_xgb = random_search_xgb.best_params_
# best_score_xgb = random_search_xgb.best_score_

# print(f"Best Hyperparameters for XGBoost: {best_params_xgb}")
# print(f"Best Cross-validation Macro F1 Score: {best_score_xgb:.4f}")

In [70]:

# best_xgb_model = random_search_xgb.best_estimator_
# y_pred_xgb_tuned = best_xgb_model.predict(X_test)


# accuracy_xgb_tuned = accuracy_score(y_test, y_pred_xgb_tuned)
# f1_macro_xgb_tuned = f1_score(y_test, y_pred_xgb_tuned, average='macro')
# conf_matrix_xgb_tuned = confusion_matrix(y_test, y_pred_xgb_tuned)
# class_report_xgb_tuned = classification_report(y_test, y_pred_xgb_tuned)

# print("Tuned XGBoost Classifier:")
# print(f"Accuracy: {accuracy_xgb_tuned:.4f}")
# print(f"Macro F1 Score: {f1_macro_xgb_tuned:.4f}")
# print("Confusion Matrix:")
# print(conf_matrix_xgb_tuned)
# print("Classification Report:")
# print(class_report_xgb_tuned)


Tuned XGBoost Classifier:
Accuracy: 0.8674
Macro F1 Score: 0.7146

In [71]:
pred = xgb_default.predict(test_pre)
original_predictions = label_encoder.inverse_transform(pred)

submission = pd.DataFrame({"id":range(0,test_pre.shape[0]),'target':original_predictions})
submission.to_csv("submission.csv",index = False)

In [72]:
model_dt = DecisionTreeClassifier(class_weight='balanced',random_state=42)

model_dt.fit(X_train, y_train)

y_dt_pred = model_dt.predict(X_test)


accuracy_dt = accuracy_score(y_test, y_dt_pred)
f1_macro_dt = f1_score(y_test, y_dt_pred, average='macro')


print(f'Accuracy: {accuracy_dt:.4f}')
print('Confusion Matrix:')
print(confusion_matrix(y_test, y_dt_pred))
print('Classification Report:')
print(classification_report(y_test, y_dt_pred))
print(f'Macro-average F1 score: {f1_macro_dt:.4f}')

Accuracy: 0.8334
Confusion Matrix:
[[6019  626]
 [ 681  517]]
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.91      0.90      6645
           1       0.45      0.43      0.44      1198

    accuracy                           0.83      7843
   macro avg       0.68      0.67      0.67      7843
weighted avg       0.83      0.83      0.83      7843

Macro-average F1 score: 0.6719


In [73]:
param_grid_dt = {
    'criterion': ['gini', 'entropy', 'log_loss'],  # Splitting criteria
    'max_depth': [None, 10, 20, 30, 50],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum samples required to split a node
    'min_samples_leaf': [1, 2, 5, 10],  # Minimum samples required to be at a leaf node
    'max_features': [None, 'sqrt', 'log2'],  # Features to consider when looking for the best split
}


grid_search_dt = GridSearchCV(
    estimator=DecisionTreeClassifier(class_weight='balanced',random_state=42),
    param_grid=param_grid_dt,
    scoring='f1_macro',  # Using macro F1 score as evaluation metric
    cv=5,  # Cross-validation splitting strategy
    verbose=1,
    n_jobs=-1  # Use all available CPU cores
)


grid_search_dt.fit(X_train, y_train)

best_params_dt = grid_search_dt.best_params_
best_score_dt = grid_search_dt.best_score_

print(f"Best Hyperparameters for Decision Tree: {best_params_dt}")
print(f"Best Cross-validation F1 Macro Score: {best_score_dt:.4f}")


best_model_dt = grid_search_dt.best_estimator_
y_dt_pred_tuned = best_model_dt.predict(X_test)


accuracy_dt_tuned = accuracy_score(y_test, y_dt_pred_tuned)
f1_macro_dt_tuned = f1_score(y_test, y_dt_pred_tuned, average='macro')

print(f'Accuracy: {accuracy_dt_tuned:.4f}')
print('Confusion Matrix:')
print(confusion_matrix(y_test, y_dt_pred_tuned))
print('Classification Report:')
print(classification_report(y_test, y_dt_pred_tuned))
print(f'Macro-average F1 score: {f1_macro_dt_tuned:.4f}')

Fitting 5 folds for each of 540 candidates, totalling 2700 fits
Best Hyperparameters for Decision Tree: {'criterion': 'entropy', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 2, 'min_samples_split': 2}
Best Cross-validation F1 Macro Score: 0.7314
Accuracy: 0.8284
Confusion Matrix:
[[5507 1138]
 [ 208  990]]
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.83      0.89      6645
           1       0.47      0.83      0.60      1198

    accuracy                           0.83      7843
   macro avg       0.71      0.83      0.74      7843
weighted avg       0.89      0.83      0.85      7843

Macro-average F1 score: 0.7432


* macro-average F1 score of 0.7432, reflecting an improvement over the baseline model's score of 0.6719.
* Precision and Recall: For Class 0 (Negative class), the model showed high precision (0.96) but lower recall (0.83), meaning it was very accurate at predicting class 0 but missed some actual class 0 instances.
* For Class 1 (Positive class), the model's recall was 0.83, but precision was lower (0.47), indicating that while the model correctly identified most of the positive instances, it also had a significant number of false positives.

In [74]:
pred = best_model_dt.predict(test_pre)
original_predictions = label_encoder.inverse_transform(pred)

submission = pd.DataFrame({"id":range(0,test_pre.shape[0]),'target':original_predictions})
submission.to_csv("submission.csv",index = False)

In [75]:
gb_model = GradientBoostingClassifier(random_state=42)

gb_model.fit(X_train, y_train)

y_gb_pred = gb_model.predict(X_test)

accuracy_gb = accuracy_score(y_test, y_gb_pred)
f1_macro_gb = f1_score(y_test, y_gb_pred, average='macro')

print(f"Accuracy: {accuracy_gb:.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_gb_pred))
print("Classification Report:")
print(classification_report(y_test, y_gb_pred))
print(f"Macro-average F1 score: {f1_macro_gb:.4f}")

Accuracy: 0.8638
Confusion Matrix:
[[6283  362]
 [ 706  492]]
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.95      0.92      6645
           1       0.58      0.41      0.48      1198

    accuracy                           0.86      7843
   macro avg       0.74      0.68      0.70      7843
weighted avg       0.85      0.86      0.85      7843

Macro-average F1 score: 0.7006


 **Class 0 (Negative Class)**:
  - **Precision**: 0.90 (The model is very accurate when predicting class 0, with only 10% of errors in its class 0 predictions).
  - **Recall**: 0.95 (95% of actual class 0 instances are correctly identified).
  - **F1-Score**: 0.92 (Good balance between precision and recall).

- **Class 1 (Positive Class)**:
  - **Precision**: 0.58 (Class 1 predictions are less accurate, with 42% of class 1 predictions being false positives).
  - **Recall**: 0.41 (The model only correctly identifies 41% of actual class 1 instances).
  - **F1-Score**: 0.48 (The model's ability to balance precision and recall for class 1 is weaker).

- **Macro-average F1 Score**: 0.70 (This average F1 score indicates the model's overall performance across both classes. It's reasonable but can be improved, especially for class 1).

In [76]:
pred = gb_model.predict(test_pre)
original_predictions = label_encoder.inverse_transform(pred)

submission = pd.DataFrame({"id":range(0,test_pre.shape[0]),'target':original_predictions})
submission.to_csv("submission.csv",index = False)

In [77]:
param_dist_gb_refined = {
    'n_estimators': [50, 100, 150, 200],  # Fewer boosting stages for quicker results
    'learning_rate': [0.05, 0.1, 0.15, 0.2],  # Focusing on a smaller range
    'max_depth': [2, 3, 4, 5],  # Limiting tree depth to prevent overfitting
    'min_samples_split': [2, 5, 10],  # Minimum samples to split a node
    'min_samples_leaf': [1, 2, 4],  # Minimum samples in a leaf
    'subsample': [0.7, 0.8, 0.9],  # Fraction of data to use for each estimator
}

# Initialize RandomizedSearchCV for Gradient Boosting
random_search_gb = RandomizedSearchCV(
    estimator=GradientBoostingClassifier(random_state=42),
    param_distributions=param_dist_gb_refined,
    n_iter=20,  # Number of parameter combinations to try
    scoring='f1_macro',  # Optimize for Macro-average F1 score
    cv=5,  # 5-fold cross-validation
    verbose=1,
    random_state=42,
    n_jobs=-1  # Use all available  cores
)


random_search_gb.fit(X_train, y_train)


best_params_gb = random_search_gb.best_params_
best_score_gb = random_search_gb.best_score_

print(f"Best Hyperparameters for Gradient Boosting (Refined): {best_params_gb}")
print(f"Best Cross-validation F1 Macro Score: {best_score_gb:.4f}")


best_model_gb = random_search_gb.best_estimator_
y_gb_pred_tuned = best_model_gb.predict(X_test)


accuracy_gb_tuned = accuracy_score(y_test, y_gb_pred_tuned)
f1_macro_gb_tuned = f1_score(y_test, y_gb_pred_tuned, average='macro')

print(f"Accuracy: {accuracy_gb_tuned:.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_gb_pred_tuned))
print("Classification Report:")
print(classification_report(y_test, y_gb_pred_tuned))
print(f"Macro-average F1 score: {f1_macro_gb_tuned:.4f}")

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best Hyperparameters for Gradient Boosting (Refined): {'subsample': 0.9, 'n_estimators': 150, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 5, 'learning_rate': 0.1}
Best Cross-validation F1 Macro Score: 0.7262
Accuracy: 0.8696
Confusion Matrix:
[[6267  378]
 [ 645  553]]
Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.94      0.92      6645
           1       0.59      0.46      0.52      1198

    accuracy                           0.87      7843
   macro avg       0.75      0.70      0.72      7843
weighted avg       0.86      0.87      0.86      7843

Macro-average F1 score: 0.7220


* The Gradient Boosting model with the tuned hyperparameters performs well with an accuracy of 86.96% and a macro F1 score of 0.72.
* It performs well on class 0 (negative class) but struggles with class 1 (positive class), as seen from the lower recall and F1 score for class 1.

In [78]:
pred = best_model_gb.predict(test_pre)
original_predictions = label_encoder.inverse_transform(pred)

submission = pd.DataFrame({"id":range(0,test_pre.shape[0]),'target':original_predictions})
submission.to_csv("submission.csv",index = False)

In [79]:
from sklearn.ensemble import AdaBoostClassifier

adaboost = AdaBoostClassifier(
    base_estimator=DecisionTreeClassifier(max_depth=1),  # Use weak learners
    n_estimators=100,  # Number of weak learners
    learning_rate=0.1,  # Learning rate shrinks the contribution of each learner
    random_state=42
)


adaboost.fit(X_train, y_train)
y_adaboost_pred = adaboost.predict(X_test)


accuracy_adaboost = accuracy_score(y_test, y_adaboost_pred)
f1_macro_adaboost = f1_score(y_test, y_adaboost_pred, average='macro')

print(f"AdaBoost Model Accuracy: {accuracy_adaboost:.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_adaboost_pred))
print("Classification Report:")
print(classification_report(y_test, y_adaboost_pred))
print(f"Macro-average F1 score: {f1_macro_adaboost:.4f}")


/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


AdaBoost Model Accuracy: 0.8560
Confusion Matrix:
[[6425  220]
 [ 909  289]]
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.97      0.92      6645
           1       0.57      0.24      0.34      1198

    accuracy                           0.86      7843
   macro avg       0.72      0.60      0.63      7843
weighted avg       0.83      0.86      0.83      7843

Macro-average F1 score: 0.6289
